# Read CGLC & ERA5land saved as Zarr @CESNET

In [1]:
pip install xarray s3fs

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import s3fs
import xarray as xr
import zarr

## Define s3 storage parameters

In [3]:
client_kwargs = {'endpoint_url': 'https://object-store.cloud.muni.cz'}
s3_path = 'Data'

In [4]:
access_key = !aws configure get aws_access_key_id
access_key = access_key[0]
secret_key = !aws configure get aws_secret_access_key
secret_key = secret_key[0]
print(access_key, secret_key)

e2d5c2e8bf9f4b3a857793176b723635 ad0478102b7142a2bd8248b6b98c13f7


In [5]:
store = s3fs.S3FileSystem(anon=True, client_kwargs = client_kwargs, key=access_key, secret=secret_key)

In [6]:
store.ls(s3_path, detail=True, refresh=True)

[{'Key': 'Data/C_GlobalLandCover_20150101_20190101_Troms-Finnmark.nc',
  'LastModified': datetime.datetime(2023, 2, 1, 12, 44, 42, 95000, tzinfo=tzutc()),
  'ETag': '"70e848cfeba6b4e2db997b6efb0ad947-8"',
  'Size': 397191332,
  'StorageClass': 'STANDARD',
  'type': 'file',
  'size': 397191332,
  'name': 'Data/C_GlobalLandCover_20150101_20190101_Troms-Finnmark.nc'},
 {'Key': 'Data/ERA5_1979-01_2021-12_Troms-Finnmark_Tmax-lweSD-Prec.nc',
  'LastModified': datetime.datetime(2023, 7, 21, 7, 8, 9, 500000, tzinfo=tzutc()),
  'ETag': '"6a41baa7f280de5fa9927b3958da4ec8-58"',
  'Size': 3038196501,
  'StorageClass': 'STANDARD',
  'type': 'file',
  'size': 3038196501,
  'name': 'Data/ERA5_1979-01_2021-12_Troms-Finnmark_Tmax-lweSD-Prec.nc'},
 {'Key': 'Data/reanalysis-era5-land_hourly_2015-01-01_2019-12-31_Troms-Finnmark_T2m-SD-TP.nc',
  'LastModified': datetime.datetime(2023, 2, 1, 9, 53, 15, 635000, tzinfo=tzutc()),
  'ETag': '"e48be2b71e08b38d296a0ea6db979b09-23"',
  'Size': 1182124070,
  'Stora

## Copernicus Global Land Cover - default chunking

In [7]:
filename = 'C_GlobalLandCover_20150101_20190101_Troms-Finnmark.zarr'
url_zarr = 's3://' + s3_path + '/' + filename
remote_zarr = zarr.storage.FSStore(url_zarr, client_kwargs=client_kwargs, key=access_key, secret=secret_key)
# remote_zarr.map.fs.find(url_zarr)

In [8]:
%%time 
GLC_default = xr.open_zarr(remote_zarr)

CPU times: user 326 ms, sys: 85.9 ms, total: 412 ms
Wall time: 3.48 s


In [9]:
%%time 
GLC_default.load()

CPU times: user 33.6 s, sys: 9.57 s, total: 43.2 s
Wall time: 3min 55s


<xarray.Dataset>
Dimensions:                             (t: 5, y: 3064, x: 16709)
Coordinates:
  * t                                   (t) datetime64[ns] 2015-01-01 ... 201...
  * x                                   (x) float64 15.6 15.6 ... 31.06 31.07
  * y                                   (y) float64 71.19 71.19 ... 68.35 68.35
Data variables: (12/17)
    Bare_CoverFraction_layer            (t, y, x) float32 255.0 255.0 ... nan
    BuiltUp_CoverFraction_layer         (t, y, x) float32 255.0 255.0 ... nan
    Change_Confidence_layer             (t, y, x) float32 255.0 255.0 ... nan
    Crops_CoverFraction_layer           (t, y, x) float32 255.0 255.0 ... nan
    DataDensityIndicator                (t, y, x) float32 255.0 255.0 ... nan
    Discrete_Classification_map         (t, y, x) float32 200.0 200.0 ... nan
    ...                                  ...
    SeasonalWater_CoverFraction_layer   (t, y, x) float32 255.0 255.0 ... nan
    Shrub_CoverFraction_layer           (t, y, x) float32 255.0 255.0 ... nan
    Snow_CoverFraction_layer            (t, y, x) float32 255.0 255.0 ... nan
    Tree_CoverFraction_layer            (t, y, x) float32 255.0 255.0 ... nan
    crs                                 |S1 b''
    dataMask                            (t, y, x) float32 nan nan ... nan nan
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform

## Copernicus Global Land Cover - chunks along the x-axis

In [10]:
filename = 'C_GlobalLandCover_20150101_20190101_Troms-Finnmark_x_2387.zarr'
url_zarr = 's3://' + s3_path + '/' + filename
remote_zarr = zarr.storage.FSStore(url_zarr, client_kwargs=client_kwargs, key=access_key, secret=secret_key)
# remote_zarr.map.fs.find(url_zarr)

In [11]:
%%time 
GLC_x_2387 = xr.open_zarr(remote_zarr)

CPU times: user 28.2 ms, sys: 4.04 ms, total: 32.2 ms
Wall time: 2.47 s


## ERA5land - default chunking

In [12]:
filename = 'reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP.zarr'
url_zarr = 's3://' + s3_path + '/' + filename
remote_zarr = zarr.storage.FSStore(url_zarr, client_kwargs=client_kwargs, key=access_key, secret=secret_key)
# remote_zarr.map.fs.find(url_zarr)

In [13]:
%%time 
ERA5land_default = xr.open_zarr(remote_zarr)

CPU times: user 24.9 ms, sys: 1.9 ms, total: 26.9 ms
Wall time: 2.9 s


## ERA5land - chunks along the time-axis

In [14]:
filename = 'reanalysis-era5-land_hourly_2015-01-01_2022-12-31_Troms-Finnmark_T2m-SD-TP_time_8766.zarr'
url_zarr = 's3://' + s3_path + '/' + filename
remote_zarr = zarr.storage.FSStore(url_zarr, client_kwargs=client_kwargs, key=access_key, secret=secret_key)
# remote_zarr.map.fs.find(url_zarr)

In [15]:
%%time 
ERA5land_time_8766 = xr.open_zarr(remote_zarr)

CPU times: user 25.3 ms, sys: 801 µs, total: 26.1 ms
Wall time: 2.98 s
